In [1]:
import json

# Load data from the JSON file
with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/output.json', 'r') as file:
    data = json.load(file)
# Preprocess the data

food_items = []
for item in data:
    food_id = item['food']['food_id']
    food_name = item['food']['food_name']
    food_type = item['food']['food_type']
    servings = item['food']['servings']['serving']

    # Process each serving entry
    for serving in servings:
        serving_description = serving['serving_description']
        calories = serving['calories']
        carbohydrates = serving['carbohydrate']
        fat = serving['fat']
        protein = serving['protein']

        # Add the processed data to the food_items list
        food_items.append({
            'food_id': food_id,
            'food_name': food_name,
            'food_type': food_type,
            'serving_description': serving_description,
            'calories': calories,
            'carbohydrates': carbohydrates,
            'fat': fat,
            'protein': protein
        })
# Print the first few food items
for item in food_items[:20]:
    print(item)


{'food_id': '2655309', 'food_name': 'Kebab', 'food_type': 'Generic', 'serving_description': '1 kebab', 'calories': '620', 'carbohydrates': '77.10', 'fat': '16.63', 'protein': '37.87'}
{'food_id': '2655309', 'food_name': 'Kebab', 'food_type': 'Generic', 'serving_description': '1 serving (390 g)', 'calories': '620', 'carbohydrates': '77.10', 'fat': '16.63', 'protein': '37.87'}
{'food_id': '2655309', 'food_name': 'Kebab', 'food_type': 'Generic', 'serving_description': '100 g', 'calories': '159', 'carbohydrates': '19.77', 'fat': '4.27', 'protein': '9.71'}
{'food_id': '2655309', 'food_name': 'Kebab', 'food_type': 'Generic', 'serving_description': '1 oz', 'calories': '45', 'carbohydrates': '5.60', 'fat': '1.21', 'protein': '2.75'}
{'food_id': '58704864', 'food_name': 'Gyoza', 'food_type': 'Brand', 'serving_description': '4 gyoza', 'calories': '120', 'carbohydrates': '17.00', 'fat': '2.50', 'protein': '7.00'}
{'food_id': '68848666', 'food_name': 'Gyoza', 'food_type': 'Brand', 'serving_descrip

In [6]:
import json
import tensorflow as tf
import tensorflow_recommenders as tfrs

# Load data from the food.json file
with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/output.json', 'r') as file:
    food_data = json.load(file)

# Load user preferences from the user.json file
with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/user_dataset.json', 'r') as file:
    user_data = json.load(file)

# Assuming user preferences are stored in a list
if isinstance(user_data, list):
    user_preferences = user_data[0]['listUserPreferences']
else:
    user_preferences = user_data['listUserPreferences']

# Preprocess the food data
food_items = []
for item in food_data:
    food_id = item['food']['food_id']
    food_name = item['food']['food_name']
    # Add other relevant features from the food.json file to the food_items list

# Preprocess the user preferences
gender = user_preferences['gender']
weight = float(user_preferences['weight'])
# Add other relevant user preferences from the user.json file

# Determine the vocabulary size based on your data
user_vocab_size = 1000  # Replace with the actual vocabulary size

# Define the desired dimensionality of the embedding space
embedding_dim = 64  # Replace with your preferred value

# Determine the vocabulary size for items based on your data
item_vocab_size = 1000  # Replace with the actual vocabulary size

# Define the collaborative filtering model
class CollaborativeFilteringModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # Define the layers for collaborative filtering
        self.embedding = tf.keras.layers.Embedding(user_vocab_size, embedding_dim)

    def call(self, inputs):
        user_embeddings = self.embedding(inputs['user_id'])
        return user_embeddings

# Define the content-based model
class ContentBasedModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # Define the layers for content-based filtering
        self.embedding = tf.keras.layers.Embedding(item_vocab_size, embedding_dim)

    def call(self, inputs):
        item_embeddings = self.embedding(inputs['item_id'])
        return item_embeddings

# Define the hybrid model
class HybridModel(tfrs.Model):
    def __init__(self):
        super().__init__()
        self.collaborative_model = CollaborativeFilteringModel()
        self.content_based_model = ContentBasedModel()
        self.dense_layer = tf.keras.layers.Dense(32, activation='relu')
        self.final_layer = tf.keras.layers.Dense(1)

    def call(self, inputs):
        user_embeddings = self.collaborative_model(inputs)
        item_embeddings = self.content_based_model(inputs)
        concatenated_embeddings = tf.concat([user_embeddings, item_embeddings], axis=1)
        x = self.dense_layer(concatenated_embeddings)
        output = self.final_layer(x)
        return output

# Define the loss function and metrics
loss = tf.keras.losses.MeanSquaredError()
metrics = [tf.keras.metrics.RootMeanSquaredError()]

# Create an instance of the hybrid model
model = HybridModel()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=loss, metrics=metrics)

# Train the model
model.fit(train_data, validation_data=val_data, epochs=10)

# Evaluate the model
results = model.evaluate(test_data)
print(f"Test RMSE: {results[1]}")

# Make recommendations
user_inputs = {'user_id': tf.constant([user_id])}  # User ID for which recommendations will be generated
item_inputs = {'item_id': tf.constant(item_ids)}  # Item IDs to consider for recommendations

recommendations = model.predict((user_inputs, item_inputs))
print(recommendations)


NameError: name 'item_vocab_size' is not defined